In [1]:
import pandas as pd
import numpy as np

In [2]:
# percent cover data
df_pc = pd.read_table('relevant/percent_cover_zipcode.txt', delimiter = ',')
df_pc.columns = ['zip', 'pc_cover']
df_pc_nyc = df_pc.loc[(df_pc['zip'] >= 10001) & (df_pc['zip'] <= 11697)]
df_cities_zips = pd.read_csv('relevant/us_cities_zips.csv')

# nyc zip codes: 10001 to 11697
df_zip_nyc = df_cities_zips.loc[(df_cities_zips['zip_code'] >= 10001) & (df_cities_zips['zip_code'] <= 11697)]
df_zip_nyc_relevant = df_zip_nyc[['zip_code', 'population', 'density']]
df_zip_nyc_relevant.columns = ['zip', 'population', 'density']

# merge datasets
combined = pd.merge(df_zip_nyc_relevant, df_pc_nyc, how="outer")
combined = combined.dropna()

In [3]:
# covid data
df_hosprate = pd.read_csv('relevant/covid/hosprate-by-modzcta.csv')
df_deathrate = pd.read_csv('relevant/covid/deathrate-by-modzcta.csv')
df_percentpos = pd.read_csv('relevant/covid/percentpositive-by-modzcta.csv')

# df_hosprate = df_hosprate.transpose()
# df_hosprate.columns = df_hosprate.iloc[0]
# df_hosprate.drop(df_hosprate.index[0])
# get covid stats [hosp rate, death rate, percent pos] averaged by zip code across all months in 2020-current
temp = []
cols = df_hosprate.columns
for i in range(7, len(cols)):
    hosp_avg = np.nanmean(list(df_hosprate[df_hosprate.columns[i]]))
    death_avg = np.nanmean(list(df_deathrate[df_deathrate.columns[i]]))
    percentpos_avg = np.nanmean(list(df_percentpos[df_percentpos.columns[i]]))
    temp.append([cols[i][-5:], hosp_avg, death_avg, percentpos_avg])
    
covid_data = pd.DataFrame(data = temp, columns = ['zip', 'hosp_avg', 'death_avg', 'percentpos_avg'])
covid_data['zip']=covid_data['zip'].astype(int)
df_combined = combined.merge(covid_data, left_on='zip', right_on='zip')